# 1. 라이브러리 로드

In [1]:
from openai import OpenAI

import pandas as pd
import numpy as np

import json

from tqdm.auto import tqdm

# 2. 데이터 로드

In [2]:
refined_district_news = pd.read_csv('bertopic_test/refined_news_topic.csv', index_col=0)
refined_district_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17075 entries, 0 to 17074
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   title                             17075 non-null  object
 1   link                              17075 non-null  object
 2   article                           17075 non-null  object
 3   date                              17075 non-null  object
 4   media                             17075 non-null  object
 5   region_code                       17075 non-null  int64 
 6   region                            17075 non-null  object
 7   district                          17075 non-null  object
 8   yyyymm                            17075 non-null  object
 9   text_length                       17075 non-null  int64 
 10  text_token_length                 17075 non-null  int64 
 11  article_summary                   17075 non-null  object
 12  preprocessed_article   

In [3]:
with open('openapi_key.json', 'r') as json_file:
    openapi_key = json.load(json_file)

client = OpenAI(api_key=openapi_key['key'])

# 감성분석
* ChatGPT API 활용
* 긍정, 부정, 중립 출현확률을 이용

In [14]:
def analyze_sentiment_korean(text, model):

    # 한글 프롬프트 작성
    prompt = f"""
        아래의 문장에 대한 감정을 분석하고, 긍정, 부정, 중립에 대한 확률을 제공해주세요.
        그리고 반드시 긍정, 부정, 중립에 대한 확률 (실수형 값)로만 아래와 같은 형식으로 반환해주세요
        무조건 아래 형식으로만 반환을 해주세요:

        긍정: [확률], 부정: [확률], 중립: [확률]

        문장: "{text}"
    """

    # OpenAI API 호출
    response = client.chat.completions.create(
        model=model,  # GPT 모델 선택
        messages=[
            {'role': 'user', 'content': prompt}
        ]
    )

    # 결과 처리
    output = response.choices[0].message.content

    return output

In [5]:
text = refined_district_news.loc[150, 'article_summary']
model = 'gpt-4o'
text

'\n 서울시가 강남구와 서초구 일대 27km2 규모의 자연녹지지역 토지를 토지거래허가구역으로 재지정했다. 부동산 투기를 사전 차단하기 위한 조치다.서울시는 12일 제6차 도시계획위원회 심의를 열고 오는 30일 만료되는 강남·서초구 일대 자연녹지를 2024년 5월 30일까지 3년 간 토지거래허가구역으로 재지정한다고 13일 밝혔다. 해당구역은 강남구 6.02km, 서초구 21.27km에 달하는 구간이다.강남구 재지정 지역은 수서역세권 공공주택지구와 구룡마을 도시개발사업이 진행되는 인접지역이다. 수서동 1.07km2를 포함해 △개포동 1.21km2 △세곡동(1.16km2) △율현동(0.54km2) △자곡동(1.25km2) △일원동(0.68km2) △대치동(0.11km2)이다.서초구는 양재R&D혁신지구와 방배동 성뒤마을 공공주택지구 등을 비롯한 양재동 1.26km2를 비롯해 △면동(2.94km2) △방배동(1.35km2) △내곡동(6.2km2) △신원동(2.09km2) △염곡동(1.45km2) △원지동(5.06km2) △서초동(0'

In [ ]:
# 감성분석 수행
refined_district_news['positive_ratio'] = np.nan
refined_district_news['neutral_ratio'] = np.nan
refined_district_news['negative_ratio'] = np.nan

for value in tqdm(refined_district_news.itertuples(), total=len(refined_district_news)):

    try:

        sentiment_raw_output = analyze_sentiment_korean(value.article_summary, model)
        sentiment_result_dic = {}
        for line in sentiment_raw_output.split(', '):
            sentiment, ratio = line.split(': ')
            sentiment_result_dic[sentiment.strip()] = float(ratio.strip())

        refined_district_news.at[value.Index, 'positive_ratio'] = sentiment_result_dic['긍정']
        refined_district_news.at[value.Index, 'negative_ratio'] = sentiment_result_dic['부정']
        refined_district_news.at[value.Index, 'neutral_ratio'] = sentiment_result_dic['중립']

    except: 
        print(f'{value.title} -> {value.link} and sentiment answer...{sentiment_raw_output}')

In [ ]:
refined_district_news_sentiment_null_df = refined_district_news[refined_district_news['positive_ratio'].isnull()]

for value in tqdm(refined_district_news_sentiment_null_df.itertuples(), total=len(refined_district_news_sentiment_null_df)):

    try:

        sentiment_raw_output = analyze_sentiment_korean(value.article_summary, model)
        sentiment_result_dic = {}
        for line in sentiment_raw_output.split(', '):
            sentiment, ratio = line.split(': ')
            sentiment_result_dic[sentiment.strip()] = float(ratio.strip())

        refined_district_news_sentiment_null_df.at[value.Index, 'positive_ratio'] = sentiment_result_dic['긍정']
        refined_district_news_sentiment_null_df.at[value.Index, 'negative_ratio'] = sentiment_result_dic['부정']
        refined_district_news_sentiment_null_df.at[value.Index, 'neutral_ratio'] = sentiment_result_dic['중립']

    except:
        print(f'{value.title} -> {value.link} and sentiment answer...{sentiment_raw_output}')

In [29]:
refined_district_news2 = pd.concat([refined_district_news[~refined_district_news['positive_ratio'].isnull()], refined_district_news_sentiment_null_df], ignore_index=True)
refined_district_news2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17075 entries, 0 to 17074
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   title                             17075 non-null  object 
 1   link                              17075 non-null  object 
 2   article                           17075 non-null  object 
 3   date                              17075 non-null  object 
 4   media                             17075 non-null  object 
 5   region_code                       17075 non-null  int64  
 6   region                            17075 non-null  object 
 7   district                          17075 non-null  object 
 8   yyyymm                            17075 non-null  object 
 9   text_length                       17075 non-null  int64  
 10  text_token_length                 17075 non-null  int64  
 11  article_summary                   17075 non-null  object 
 12  prep

In [32]:
refined_district_news2.isnull().sum()

title                                0
link                                 0
article                              0
date                                 0
media                                0
region_code                          0
region                               0
district                             0
yyyymm                               0
text_length                          0
text_token_length                    0
article_summary                      0
preprocessed_article                 0
preprocessed_article_summary         0
article_len                          0
article_summary_len                  0
preprocessed_article_len             0
preprocessed_article_summary_len     0
topic                                0
Topic                                0
Name                                 0
Representation                       0
positive_ratio                      78
neutral_ratio                       78
negative_ratio                      78
dtype: int64

In [ ]:
# refined_district_news2.to_csv('refined_news_topic_sentiment.csv', encoding='utf-8-sig')